In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import json
import time
import datetime as dt
import re

In [2]:
# Dealers description
dealer_dct = {
    'id': 'simpleautos',
    'name': 'Simple Auto',
    'street': '2829 Derry Rd E.',
    'city': 'Mississauga',
    'zip': 'L4T1A5',
    'province': 'ON',
    'phone': '905-965-7800',
    'url': 'https://www.simpleautos.ca',
    'latitude': 43.701760,
    'longitude': -79.645490
}

print(json.dumps(dealer_dct, indent=4))

with open('data/simpleautos.json', 'w', encoding='utf-8') as fp:
    fp.writelines(json.dumps(dealer_dct) + "\n")

{
    "id": "simpleautos",
    "name": "Simple Auto",
    "street": "2829 Derry Rd E.",
    "city": "Mississauga",
    "zip": "L4T1A5",
    "province": "ON",
    "phone": "905-965-7800",
    "url": "https://www.simpleautos.ca",
    "latitude": 43.70176,
    "longitude": -79.64549
}


In [3]:
base_url ='https://www.simpleautos.ca'
inventory_url = 'https://www.simpleautos.ca/used-cars'

In [4]:
# Function uses inventory url (defined above), takes url parameters (such as a page number) as a dict,
# parses the inventory page and returns a list of URLs to car description pages
# Parsing stops when first sold vehicle found or when no search results found on a page
# The function returns a list or URLs and a boolean: True if there might be more data, False when done
def parse_inventory(query_args):
    global base_url, inventory_url
    res = []

    time.sleep(1)
    response = requests.get(inventory_url, query_args)
    
    soup = BeautifulSoup(response.text, 'lxml')
    vehicles = soup.find_all('div', class_='vehicle search-result-item vehicleList')
    if not vehicles: # empty list
        return res, False
    
    for v in vehicles:
        # check if it isn't sold
        sold = v.find('div', class_='SoldVehicle')
        if sold is not None:
            return res, False # all the rest of the inventory is a list of sold vehicles, no need to go further

        # get vehicle description page url
        url = base_url + v.div.contents[1].div.div.a['href']
        
        # Printing URL (for debugginh purposes)
        print(url)
        res.append(url)
    
    return res, True

In [5]:
# Making a list of inventory URLs
vehicle_urls = []
query_args = {'ppage': 50, 'cpage': 1}

urls, has_more = parse_inventory(query_args)

while has_more and urls:
    vehicle_urls.extend(urls)
    query_args['cpage'] += 1
    urls, has_more = parse_inventory(query_args)
    
vehicle_urls.extend(urls)
print(len(vehicle_urls))

https://www.simpleautos.ca/cars/used/2014-Chevrolet-Cruze-Sedan-114980
https://www.simpleautos.ca/cars/used/2017-Volkswagen-Jetta-Sedan-Sedan-114748
https://www.simpleautos.ca/cars/used/2019-Volkswagen-Jetta-Sedan-114737
https://www.simpleautos.ca/cars/used/2015-Mazda-MAZDA3-Sedan-114734
https://www.simpleautos.ca/cars/used/2013-Nissan-Altima-Sedan-114169
https://www.simpleautos.ca/cars/used/2019-Hyundai-Elantra-Sedan-114162
https://www.simpleautos.ca/cars/used/2018-Honda-Civic-Sedan-Sedan-114159
https://www.simpleautos.ca/cars/used/2016-Volkswagen-Passat-Sedan-114156
https://www.simpleautos.ca/cars/used/2018-Toyota-Corolla-Sedan-114153
https://www.simpleautos.ca/cars/used/2017-Honda-Accord-Sedan-Sedan-111421
https://www.simpleautos.ca/cars/used/2017-Honda-Accord-Sedan-Sedan-110440
https://www.simpleautos.ca/cars/used/2010-Ford-F-150-Truck-Extended-Cab-110187
https://www.simpleautos.ca/cars/used/2016-Volkswagen-Jetta-Sedan-Sedan-106068
https://www.simpleautos.ca/cars/used/2017-Hyundai-

In [8]:
# Function parses vehicle description page and returns a dictionary with its parameters
def parse_car_infopage(url):
    # Translation table from their names to our key names
    translation_table = {
        'Make:': 'make',
        'Model:': 'model',
        'Year:': 'year',
        'Body Style:': 'body_style',
        'Odometer:': 'mileage', # Note: special treatment
        'Transmission:': 'transmission',
        'Engine:': 'cylinders', # Note: special treatment
        'Engine Size:': 'displacement',
        'Driveline:': 'drivetrain',
        'Exterior Color:': 'ext_color', 
        'Interior Color:': 'int_color',
        'Doors:': 'doors', # Note: special treatment
        'Passengers:': 'passengers',
        'Stock Number:': 'stock_id',
        'VIN:': 'vin'
    }
    
    # Fetching the page
    time.sleep(1)
    response = requests.get(url)

    # Parsing
    soup = BeautifulSoup(response.text, 'lxml')
    price_node = soup.find('span', class_='PriceValue')
    details_node = soup.find('div', class_='VehicleInfoDetails')
    features_node = soup.find('ul', class_='VehicleOptions')

    # Basic info
    res = {
        'timestamp': dt.datetime.now().isoformat(),
        'dealer': 'simpleautos',
        'url': url
    }

    if (price_node is None) or (details_node is None) or (features_node is None):
        res["invalid"] = "invalid"
        return res

    # Price
    try:
        price = re.sub('[^0-9]', '', price_node.contents[0]) # Cleaning junk, leaving only numbers
        res['price'] = int(price)
    except:
        res["invalid"] = "invalid"
        return res

    # Reading core parameters
    spans = details_node.find_all('span')

    for s in spans:
        if s.text in translation_table:
            key = translation_table[s.text]
            value = s.next_sibling.text

            try:
                # Ad Hoc treatment and type conversion
                if key in ['cylinders', 'doors']:
                    value = int(value.split()[0])
                elif key == 'mileage':
                    value = int(re.sub('[^0-9]', '', value)) # Remove all non numbers using regexp substitution
                elif key in ['year', 'passengers']:
                    value = int(value) # some passenger number info have value "-"
                elif key in ['displacement']:
                    value = float(value)
            except:
                key = ''

            if key:
                res[key] = value
    
    # Features and options
    res['features'] = []
    items = features_node.find_all('li')
    for i in items:
        res['features'].append(i.text)
    
    # Printing result (for debugging purposes)
    print(json.dumps(res, indent=4))
    
    # Done
    return res

In [9]:
# Now scraping vehicle data, one by one
vehicle_data = []

for url in vehicle_urls:
    res = parse_car_infopage(url)
    if 'invalid' not in res:
        vehicle_data.append(res)
    
print(len(vehicle_data))

with open('data/simpleautos_cars.json', 'w', encoding='utf-8') as fp:
    for v in vehicle_data:
        fp.writelines(json.dumps(v) + "\n")

{
    "timestamp": "2020-04-30T16:52:02.608984",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca/cars/used/2014-Chevrolet-Cruze-Sedan-114980",
    "price": 6499,
    "make": "Chevrolet",
    "model": "Cruze",
    "year": 2014,
    "body_style": "Sedan",
    "mileage": 152320,
    "transmission": "Automatic",
    "cylinders": 4,
    "displacement": 1.4,
    "drivetrain": "FWD",
    "ext_color": "Black",
    "int_color": "Black",
    "doors": 4,
    "passengers": 5,
    "stock_id": "616",
    "vin": "1G1PC5SB5E7246616",
    "features": [
        "Air Conditioning",
        "Alloy Wheels",
        "AM/FM Stereo",
        "Backup Camera",
        "Bluetooth",
        "CD Player",
        "Child-Safety Locks",
        "Cloth Interior",
        "Cruise Control",
        "Cup Holder",
        "Daytime Running Lights",
        "Entertainment System",
        "Heated Exterior Mirrors",
        "Keyless Entry",
        "Low Tire Pressure Warning",
        "Power Brakes",
    